# Markdown Source

In [1]:
%%writefile tests/demo.md
---
amsthm:
  plain:
  - Theorem: [Lemma, Corollary]
  - With Space*
  - Proposition: Conjecture
  - WithoutSpace
  definition:
  - Definition
  remark:
  - Case
  parentcounter:
  - chapter
...

<div class="Theorem" info="within parenthesis">
plain theoremstyle here
</div>

<div class="Theorem" id="simplestEquation"}
Label and reference:

$$E=mc^2$$
</div>

From the \ref{simplestEquation}, we see that...

<div class="With Space" info="within parenthesis">
Environment name has a space, and is unnumbered.
</div>

<div class="Lemma" info="within parenthesis">
This one share counter with Theorem
</div>

<div class="Definition" info="within parenthesis">
definition theoremstyle here
</div>

<div class="Case" info="within parenthesis">
remark theoremstyle here
</div>

<div class="proof" info="Proof of the Main Theorem">
Predefined proof theoremstyle here
</div>


Overwriting tests/demo.md


## Shorter Markdown Source

## In native

In [2]:
!pandoc -s -t native tests/demo.md

Pandoc (Meta {unMeta = fromList [("amsthm",MetaMap (fromList [("definition",MetaList [MetaInlines [Str "Definition"]]),("parentcounter",MetaList [MetaInlines [Str "chapter"]]),("plain",MetaList [MetaMap (fromList [("Theorem",MetaList [MetaInlines [Str "Lemma"],MetaInlines [Str "Corollary"]])]),MetaInlines [Str "With",Space,Str "Space*"],MetaMap (fromList [("Proposition",MetaInlines [Str "Conjecture"])]),MetaInlines [Str "WithoutSpace"]]),("remark",MetaList [MetaInlines [Str "Case"]])]))]})
[Div ("",["Theorem"],[("info","within parenthesis")])
 [Para [Str "plain",Space,Str "theoremstyle",Space,Str "here"]]
,RawBlock (Format "html") "<div class=\"Theorem\" id=\"simplestEquation\"}\nLabel and reference:\n\n$$E=mc^2$$\n</div>\n"
,Para [Str "From",Space,Str "the",Space,RawInline (Format "tex") "\\ref{simplestEquation}",Str ",",Space,Str "we",Space,Str "see",Space,Str "that..."]
,Div ("",["With","Space"],[("info","within parenthesis")])
 [Para [Str "Environment",Space,Str "name",Space,Str "h

## In JSON

In [3]:
!pandoc -s -t json tests/demo.md

{"blocks":[{"t":"Div","c":[["",["Theorem"],[["info","within parenthesis"]]],[{"t":"Para","c":[{"t":"Str","c":"plain"},{"t":"Space"},{"t":"Str","c":"theoremstyle"},{"t":"Space"},{"t":"Str","c":"here"}]}]]},{"t":"RawBlock","c":["html","<div class=\"Theorem\" id=\"simplestEquation\"}\nLabel and reference:\n\n$$E=mc^2$$\n</div>\n"]},{"t":"Para","c":[{"t":"Str","c":"From"},{"t":"Space"},{"t":"Str","c":"the"},{"t":"Space"},{"t":"RawInline","c":["tex","\\ref{simplestEquation}"]},{"t":"Str","c":","},{"t":"Space"},{"t":"Str","c":"we"},{"t":"Space"},{"t":"Str","c":"see"},{"t":"Space"},{"t":"Str","c":"that..."}]},{"t":"Div","c":[["",["With","Space"],[["info","within parenthesis"]]],[{"t":"Para","c":[{"t":"Str","c":"Environment"},{"t":"Space"},{"t":"Str","c":"name"},{"t":"Space"},{"t":"Str","c":"has"},{"t":"Space"},{"t":"Str","c":"a"},{"t":"Space"},{"t":"Str","c":"space,"},{"t":"Space"},{"t":"Str","c":"and"},{"t":"Space"},{"t":"Str","c":"is"},{"t":"Space"},{"t":"Str","c":"unnumbered."}]}]]},{"t":"

# panflute testing filter

In [4]:
import panflute as pf
print(pf.__version__)
# pf.Doc.get_metadata?

1.9.6


In [5]:
%%writefile test.py
#!/usr/bin/env python3
"""
Pandoc filter using panflute
"""

from collections import OrderedDict, ChainMap
import panflute as pf
import json


def parse_metadata(eachStyle):
    """
    Input: eachStyle is the metadata under the key of amsthm styles, i.e. plain, definition, or remark
    Output:
    environments: set of all environments
    unnumbered: set of all unnumbered environments
    counter_dict: a dictionary with keys as the environments and the values as the shared counter
    counter: a dictionary with keys as the shared counter, and values as the counter (int), initialized as 0
    shared_environments: a dictionary with keys as the shared counter, and values as all other environments using this counter
    """
    environments = set()
    unnumbered = set()
    counter_dict = {}
    counter = {}
    shared_environments = {}
    for environment in eachStyle:
        # check if it is a dict: hence numbered environments with shared counters
        if isinstance(environment, dict):
            for main, shared in environment.items():
                environments.add(main)
                counter_dict[main] = main
                counter[main] = 0
                # check if it is a string (1 item) or a list
                if isinstance(shared, str):
                    environments.add(shared)
                    counter_dict[shared] = main
                    shared_environments[main] = [shared]
                else:
                    environments.update(shared)
                    for i in shared:
                        counter_dict[i] = main
                    shared_environments[main] = shared
        # check if it is unnumbered
        elif str(environment)[-1] == '*':
            environment = str(environment)[0:-1]
            environments.add(environment)
            unnumbered.add(environment)
        # else it is a numbered environment with unshared counter
        else:
            environment = str(environment)
            environments.add(environment)
            counter_dict[environment] = environment
            counter[environment] = 0
    return environments, unnumbered, counter_dict, counter, shared_environments


def get_metadata(doc):
    """
    Getting the metadata:

    - doc.environments: set of all environments for matching the div
    - doc.unnumbered: set of unnumbered environments for checking if counter is needed
    - doc.counter_dict: dict to lookup the shared counter for each environment
    - doc.counter: dict of shared counters
    - doc.style: dicts of plain, definition, remark
    - header level mapping to part/chapter/section @todo
    """
    amsthm_metadata = doc.get_metadata('amsthm')
    all_parsed_metadata = {}
    amsthm_style = ('plain', 'definition', 'remark')
    for i in amsthm_style:
        all_parsed_metadata[i] = parse_metadata(amsthm_metadata[i])
    # (0, 1, 2, 3, 4) corresponds to (environments, unnumbered, counter_dict, counter, shared_environments)
    doc.environments = set().union(*[all_parsed_metadata[i][0] for i in amsthm_style])
    doc.unnumbered = set().union(*[all_parsed_metadata[i][1] for i in amsthm_style])
    doc.counter_dict = dict(ChainMap(*[all_parsed_metadata[i][2] for i in amsthm_style]))
    doc.counter = dict(ChainMap(*[all_parsed_metadata[i][3] for i in amsthm_style]))
    doc.shared_environments = dict(ChainMap(*[all_parsed_metadata[i][4] for i in amsthm_style]))
    doc.style = {i: all_parsed_metadata[i][0] for i in amsthm_style}
    

def amsthm(elem, doc):
    print('doc.environments:', doc.environments)
    print('doc.unnumbered:', doc.unnumbered)
    print('doc.counter_dict:', doc.counter_dict)
    print('doc.counter:', doc.counter)
    print('doc.shared_environments:', doc.shared_environments)
    print('doc.style:', doc.style)
#     print(doc.get_metadata(''))
#     k,v = dict(doc.get_metadata('amsthm.plain'))
#     print(k, v)
#     for i in doc.get_metadata('amsthm'):
#         print(i)
#     for i in doc.content:
#         print(type(i))
#     if isinstance(elem, pf.MetaMap) and 'amsthm' in elem.content:
#         elem.content['amsthm'].walk(print, doc)

#     if isinstance(elem, pf.MetaMap):
#         print(elem)
#         print(elem.next)
#     if isinstance(elem, pf.MetaMap):
#         print(elem)
    pass


def finalize(doc):
    pass


def main(doc=None):
    return pf.run_filter(amsthm,
                         prepare=get_metadata,
                         finalize=finalize,
                         doc=doc) 


if __name__ == '__main__':
    main()

Overwriting test.py


In [6]:
!chmod +x test.py

## Output

In [7]:
!pandoc -s -t json tests/demo.md | ./test.py

doc.environments: {'Proposition', 'With Space', 'Corollary', 'Theorem', 'Definition', 'Conjecture', 'Case', 'WithoutSpace', 'Lemma'}
doc.unnumbered: {'With Space'}
doc.counter_dict: {'Proposition': 'Proposition', 'Corollary': 'Theorem', 'Definition': 'Definition', 'Conjecture': 'Proposition', 'Case': 'Case', 'Theorem': 'Theorem', 'WithoutSpace': 'WithoutSpace', 'Lemma': 'Theorem'}
doc.counter: {'Proposition': 0, 'Theorem': 0, 'WithoutSpace': 0, 'Definition': 0, 'Case': 0}
doc.shared_environments: {'Proposition': ['Conjecture'], 'Theorem': ['Lemma', 'Corollary']}
doc.style: {'plain': {'With Space', 'Corollary', 'Theorem', 'Conjecture', 'Lemma', 'Proposition', 'WithoutSpace'}, 'remark': {'Case'}, 'definition': {'Definition'}}
doc.environments: {'Proposition', 'With Space', 'Corollary', 'Theorem', 'Definition', 'Conjecture', 'Case', 'WithoutSpace', 'Lemma'}
doc.unnumbered: {'With Space'}
doc.counter_dict: {'Proposition': 'Proposition', 'Corollary': 'Theorem', 'Definition': 'Definition', '

# YAML Test

In [8]:
YAML = """
amsthm:
  plain:
  - Theorem: Lemma
  - With Space*
  definition:
  - Definition
  remark:
  - Case
  parentcounter:
  - chapter
"""

In [9]:
import yaml

In [10]:
options = yaml.load(YAML)
print(options)

{'amsthm': {'remark': ['Case'], 'parentcounter': ['chapter'], 'plain': [{'Theorem': 'Lemma'}, 'With Space*'], 'definition': ['Definition']}}


In [11]:
options['amsthm']

{'definition': ['Definition'],
 'parentcounter': ['chapter'],
 'plain': [{'Theorem': 'Lemma'}, 'With Space*'],
 'remark': ['Case']}